## The basics

OCaml is very similar to other languages where basic types are concerned.

In [1]:
1

- : int = 1


`: T = V` means that the expression we evaluated to value `V` and has type `T`.   OCaml uses _type inference_ to automatically figure out the type of an expression.  `int` is the type of an integer.

In [2]:
"Hello world!"

- : string = "Hello world!"


In [3]:
1 + 2

- : int = 3


OCaml also has a strong type system that will let you know when you do something stupid.

In [4]:
1 + "String"

error: compile_error

Let's look at a more complex expression such as a list.

In [5]:
[1; 2; 3]

- : int list = [1; 2; 3]


Our list has type `int list`.  You can think of this as a `list` of `int`s.

What type does the empty list `[]` have?

In [6]:
[]

- : 'a list = []


`'a` is a _type_ variable.  This indicates that `[]` is a list of elements of some type that is yet to be determined.

## Functions

In functional languages, functions are expressions.

In [7]:
fun arg -> 42

- : 'a -> int = <fun>


Let's unpack this.  We got back a value of type `'a -> int`, which is a function that takes an undetermined value as an argument, and returns an integer.

Our function does not have a name; it's anonymous.  Many languages call these _lambda functions_.

Let's give our function a name!

In [8]:
let myfun = fun arg -> 42

val myfun : 'a -> int = <fun>


In [9]:
myfun

- : 'a -> int = <fun>


Our function is now named `myfun`.

In [10]:
myfun 0

- : int = 42


In [11]:
myfun 0.0

- : int = 42


In [12]:
myfun ["Wow this is crazy"]

- : int = 42


We can also use a slightly shorter syntax when we are defining functions.

In [13]:
let myfun arg = 42

val myfun : 'a -> int = <fun>


In [14]:
let succ n = n + 1

val succ : int -> int = <fun>


Look at that.  Now we have a function that takes an integer argument and returns an integer.  OCaml's type inference figured out the argument and return types are `int` because they are being added.  (Floats have their own addition operator `+.`.)

In [15]:
succ 42

- : int = 43


In [16]:
let id a = a

val id : 'a -> 'a = <fun>


We just defined a function that takes an undetermined type and returns the same type!

In [17]:
id 0

- : int = 0


In [18]:
id "You won't return this value"

- : string = "You won't return this value"


In [19]:
id [myfun]

- : ('_weak1 -> int) list = [<fun>]


Let's define a function that takes two arguments.

In [20]:
let add a b = a + b

val add : int -> int -> int = <fun>


- We got back a crazy type `int -> int -> int`
- The trick is to add parentheses like  `int -> (int -> int)`
- If you provide an `int`, you get back a new function that takes an `int`
- This idea is called _currying_

Just like we would expect, we can pass both arguments at once.

In [21]:
add 40 2

- : int = 42


But because of currying, we can pass a single argument and we'll still get back a function!  This is called _partial application_.

In [22]:
add 40

- : int -> int = <fun>


And we can save it and call it as usual.

In [23]:
let add1 = add 1;;
add1 41

val add1 : int -> int = <fun>


- : int = 42


## Fun with Lists

### map

Having first-class functions allows us to write a lot of interesting list processing functions.  Let's start with `List.map`.  OCaml organizes code into modules.  `List.map` simply means the `map` symbol in the `List` module.  In this case, `map` is a function.

`map f [e0; e1; ...; en]` is defined to be `[f e0; f e1; ...; f en]`.  In other words, `map` applies the function `f` to every element of the list to create a new list.  Notice that we provide a function `f` as an argument to `map`; this is possible because functions are first class citizens.  This makes `map` a _higher order function_.

Let's try a few examples!

In [24]:
let add42 x = x + 42;;
List.map add42 [1; 2; 3]

val add42 : int -> int = <fun>


- : int list = [43; 44; 45]


Lambda functions can be used to succinctly describe `f` without naming it.

In [25]:
List.map (fun x -> x + 42) [1; 2; 3]

- : int list = [43; 44; 45]


In [26]:
let square x = x*x;;
List.map square [0; 1; 2; 3; 4]

val square : int -> int = <fun>


- : int list = [0; 1; 4; 9; 16]


In [27]:
let l1 = [0; 1; 2; 3; 4];;
let l2 = List.map square l1;;
let l3 = List.map square l2;;

val l1 : int list = [0; 1; 2; 3; 4]


val l2 : int list = [0; 1; 4; 9; 16]


val l3 : int list = [0; 1; 16; 81; 256]


### Filter

Another very useful function is `List.filter`.  `filter f l` runs `f` on each element of `l`, and returns a list consisting solely of the elements of `l` that `f` returned true on.  Unlike with `map`, `f` must return a boolean when used with `filter`.

In [28]:
List.filter

- : ('a -> bool) -> 'a list -> 'a list = <fun>


In this case, `f` has type `'a -> bool` because it's deciding yes (true) or no (false) for each element of the list.

Let's try a few examples.

In [29]:
let is_positive x = x > 0;;
List.filter is_positive [-2; -1; 0; 1; 2]

val is_positive : int -> bool = <fun>


- : int list = [1; 2]


In [30]:
let is_even x = x mod 2 == 0;;
List.filter is_even [1; 2; 3; 4; 5; 6; 7; 8; 9; 10]

val is_even : int -> bool = <fun>


- : int list = [2; 4; 6; 8; 10]


## Flattening

Sometimes we need to map one object to multiple other cases.  One way to do this is to map each element to a list.  

In [31]:
List.map (fun x -> [x; x+1; x+2]) [1;2;3]

- : int list list = [[1; 2; 3]; [2; 3; 4]; [3; 4; 5]]


We got back a `list` of `list`s of `int`s.  But how can we turn this into a `list` of `int`s?  We can use `List.flatten`.  `flatten [[e11; e12]; ...; [en1; en2]]` returns `[e11; e12; ...; en1; en2]`.

In [32]:
let x = List.map (fun x -> [x; x+1; x+2]) [1;2;3];;
List.flatten x

val x : int list list = [[1; 2; 3]; [2; 3; 4]; [3; 4; 5]]


- : int list = [1; 2; 3; 2; 3; 4; 3; 4; 5]


### Folding

Our favorite functional list operation is called _folding_.  The intuition of folding is that you take a list of values and combine them into a single value.  Another way to think about folding is that it _accumulates_ the information in a list to a single value.

For example, we usually think of addition as a binary operation.  But we can use `fold` to apply addition to a list.  For example, if we folded addition over `[1;2;3]` starting with `0`, we would intuitively get `((0+1)+2)+3`, the sum of all elements in the list.  The sum we're keeping track of is called the _accumulator_.  It starts with `0`, becomes `1`, then `3`, and then `6`.

In [33]:
let add accum n = accum + n;;
let sum l = List.fold_left add 0 l;;
sum [1;2;3;4]

val add : int -> int -> int = <fun>


val sum : int list -> int = <fun>


- : int = 10


Here we're using `fold` to compute the sum of a list of numbers just like we talked about.

`(((0+1)+2)+3)+4 = 10`

Notice that the first argument of `add` is `accum`, the accumulated value so far.

In [34]:
let mult accum n = accum * n;;
let product l = List.fold_left mult 1 l;;
product [1;2;3;4]

val mult : int -> int -> int = <fun>


val product : int list -> int = <fun>


- : int = 24


Here we're folding over multiplication to compute the product of a list.  Note that we have to use `1` as our initial accumulator, since `0*n = 0`!

`(((1*1)*2)*3)*4 = 24`

Ok, let's try to write a function that computes the length of a list.

In [35]:
let increase_count accum element = accum + 1;;
let list_length list = List.fold_left increase_count 0 list

val increase_count : int -> 'a -> int = <fun>


val list_length : 'a list -> int = <fun>


There's a lot going on here!  We're going to _accumulate_ the number of list elements we've seen so far.  Our `increase_count` function takes the old accumulator and the next list element and returns the new accumulator.  But since we're just counting the number of elements, we actually don't need to use the list element at all.  So we just take the old accumulator and add one.

We also set `0` as the initial accumulator.  That's because we're counting, and before we have seen any list elements, the count is 0.

In [36]:
list_length [1; 2; 3]

- : int = 3


In [37]:
list_length []

- : int = 0


Folding over an empty list `[]` simply returns the initial accumulator value.

Ok, let's do one more trick.  Let's use `fold` to reverse a list.  Our accumulator will actually hold the reversed list.  We'll start with an empty list.  Each time we visit a list element, we'll put it on the front of the accumulated list.  So the first element will be at the end of the list, and the last element will be at the front.

`a :: b` means to put `a` at the front of list `b`.  For example, `1::[2;3] = [1;2;3]`.

In [38]:
let add_to_front_of_list accum element = element :: accum;;
let reverselist l = List.fold_left add_to_front_of_list [] l

val add_to_front_of_list : 'a list -> 'a -> 'a list = <fun>


val reverselist : 'a list -> 'a list = <fun>


In [39]:
reverselist [1; 2; 3]

- : int list = [3; 2; 1]


Our function is actually computing `3::(2::(1::[])) = [3;2;1]`.

## A Fun Example

From [Advent of Code](https://adventofcode.com/2020/day/1)

Specifically, they need you to find the two entries that sum to 2020 and then multiply those two numbers together.
 
For example, suppose your expense report contained the following:
 
 
    1721
    979
    366
    299
    675
    1456

In this list, the two entries that sum to 2020 are 1721 and 299. Multiplying them together produces 1721 * 299 = 514579, so the correct answer is 514579.

So how can we solve this functionally?

1. Read the numbers into a list of strings: `["1";"2";"3"]`
2. Convert to a list of integers with `map`: `[1;2;3]`
3. Build a list of all tuples using `map` and `flatten`: `[(1,1);(1,2);(1,3);(2,1);(2,2);(2,3);(3,1);(3,2);(3,3)]`
4. Find a tuple that sums to 2020 using `find`: `(n1,n2)`
5. Multiply it!

We won't have time to go through this example in detail, but we worked through it step by step in the notebook.  Check it out!

Now we'll look at using some functional capabilities in javascript and python.

In [40]:
let input_string = {foo|1865
1179
1328
1390
322
1999
1713
1808
1380
1727
1702
1304
1481
1334
1728
1953
1413
1753
1723
1379
1532
1918
1490
71
1388
1519
807
1427
1729
1952
970
1405
1500
1782
1899
1501
1720
1832
1706
1658
1333
486
1670
1674
1290
1893
1382
1761
1945
1607
1319
1508
1464
1733
1917
1483
1620
1677
1835
1810
1385
1840
1705
1994
1695
1599
1681
1566
1153
1672
1373
1679
1714
1283
1950
1197
1696
1936
1218
1910
1786
958
1565
1583
1914
1853
1682
1267
1543
1322
1965
1406
860
1754
1867
1393
1404
1191
1861
2007
1613
1938
1340
1227
1628
1826
1638
1970
1993
1748
496
1661
1736
1593
1298
1882
1763
1616
1848
92
1338
1707
1587
1996
1708
700
1460
1673
1450
1815
1537
1825
1683
1743
1949
1933
1289
1905
1307
1845
1855
1955
1554
1570
1931
1780
1929
1980
1978
1998
1371
1981
1817
1722
1545
1561
1352
1935
1553
1796
1847
1640
1414
1198
1669
1909
1879
1744
1783
1367
1632
1990
1937
1919
1431
2002
1603
1948
1317
1282
1349
1839
1575
1730
1849
1959
1971
2009
1641
1402
1924
1757
1605
1693
1762
283
1525
1964
1715
1602|foo}

val input_string : string =
  "1865\n1179\n1328\n1390\n322\n1999\n1713\n1808\n1380\n1727\n1702\n1304\n1481\n1334\n1728\n1953\n1413\n1753\n1723\n1379\n1532\n1918\n1490\n71\n1388\n1519\n807\n1427\n1729\n1952\n970\n1405\n1500\n1782\n1899\n1501\n1720\n1832\n1706\n1658\n1333\n486\n1670\n1674\n1290\n1893\n1382\n1761\n1945\n1607\n1319\n1508\n1464\n1733\n1917\n1483\n1620\n1677\n1835\n1810\n1385\n"... (* string length 986; truncated *)


This is our starting problem input.

### 1: Read the numbers into a list of strings


In [41]:
let l = Str.split (Str.regexp "\n") input_string

val l : string list =
  ["1865"; "1179"; "1328"; "1390"; "322"; "1999"; "1713"; "1808"; "1380";
   "1727"; "1702"; "1304"; "1481"; "1334"; "1728"; "1953"; "1413"; "1753";
   "1723"; "1379"; "1532"; "1918"; "1490"; "71"; "1388"; "1519"; "807";
   "1427"; "1729"; "1952"; "970"; "1405"; "1500"; "1782"; "1899"; "1501";
   "1720"; "1832"; "1706"; "1658"; "1333"; "486"; "1670"; "1674"; "1290";
   "1893"; "1382"; "1761"; "1945"; "1607"; "1319"; "1508"; "1464"; "1733";
   "1917"; "1483"; "1620"; "1677"; "1835"; "1810"; "1385"; "1840"; "1705";
   "1994"; "1695"; "1599"; "1681"; "1566"; "1153"; "1672"; "1373"; "1679";
   "1714"; "1283"; "1950"; "1197"; "1696"; "1936"; "1218"; "1910"; "1786";
   "958"; "1565"; "1583"; "1914"; "1853"; "1682"; "1267"; "1543"; "1322";
   "1965"; "1406"; "860"; "1754"; "1867"; "1393"; "1404"; "1191"; "1861";
   "2007"; "1613"; "1938"; "1340"; "1227"; "1628"; "1826"; "1638"; "1970";
   "1993"; "1748"; "496"; "1661"; "1736"; "1593"; "1298"; "1882"; "1763";
   "1616"; "

### 2: Convert to a list of integers with `map`


In [42]:
let l2 = List.map int_of_string l

val l2 : int list =
  [1865; 1179; 1328; 1390; 322; 1999; 1713; 1808; 1380; 1727; 1702; 1304;
   1481; 1334; 1728; 1953; 1413; 1753; 1723; 1379; 1532; 1918; 1490; 71;
   1388; 1519; 807; 1427; 1729; 1952; 970; 1405; 1500; 1782; 1899; 1501;
   1720; 1832; 1706; 1658; 1333; 486; 1670; 1674; 1290; 1893; 1382; 1761;
   1945; 1607; 1319; 1508; 1464; 1733; 1917; 1483; 1620; 1677; 1835; 1810;
   1385; 1840; 1705; 1994; 1695; 1599; 1681; 1566; 1153; 1672; 1373; 1679;
   1714; 1283; 1950; 1197; 1696; 1936; 1218; 1910; 1786; 958; 1565; 1583;
   1914; 1853; 1682; 1267; 1543; 1322; 1965; 1406; 860; 1754; 1867; 1393;
   1404; 1191; 1861; 2007; 1613; 1938; 1340; 1227; 1628; 1826; 1638; 1970;
   1993; 1748; 496; 1661; 1736; 1593; 1298; 1882; 1763; 1616; 1848; 92; 1338;
   1707; 1587; 1996; 1708; 700; 1460; 1673; 1450; 1815; 1537; 1825; 1683;
   1743; 1949; 1933; 1289; 1905; 1307; 1845; 1855; 1955; 1554; 1570; 1931;
   1780; 1929; 1980; 1978; 1998; 1371; 1981; 1817; 1722; 1545; 1561; 1352;
   1935; 15

### 3: Turn the list into a list of all tuples with `map` and `flatten`

This is the only tricky part.  We'll first write a function `make_tuples n1` that takes a number `n1` and makes a tuple for each input where the first tuple is `n1` and the the second is the input.

In [43]:
let make_tuples n1 = List.map (fun n2 -> (n1, n2)) l2;;

val make_tuples : 'a -> ('a * int) list = <fun>


And now we map over `l` using our new function!

In [44]:
let l3 = List.map make_tuples l2

val l3 : (int * int) list list =
  [[(1865, 1865); (1865, 1179); (1865, 1328); (1865, 1390); (1865, 322);
    (1865, 1999); (1865, 1713); (1865, 1808); (1865, 1380); (1865, 1727);
    (1865, 1702); (1865, 1304); (1865, 1481); (1865, 1334); (1865, 1728);
    (1865, 1953); (1865, 1413); (1865, 1753); (1865, 1723); (1865, 1379);
    (1865, 1532); (1865, 1918); (1865, 1490); (1865, 71); (1865, 1388);
    (1865, 1519); (1865, 807); (1865, 1427); (1865, 1729); (1865, 1952);
    (1865, 970); (1865, 1405); (1865, 1500); (1865, 1782); (1865, 1899);
    (1865, 1501); (1865, 1720); (1865, 1832); (1865, 1706); (1865, 1658);
    (1865, 1333); (1865, 486); (1865, 1670); (1865, 1674); (1865, 1290);
    (1865, 1893); (1865, 1382); (1865, 1761); (1865, 1945); (1865, 1607);
    (1865, 1319); (1865, 1508); (1865, 1464); (1865, 1733); (1865, 1917);
    (1865, 1483); (1865, 1620); (1865, 1677); (1865, 1835); (1865, 1810);
    (1865, 1385); (1865, 1840); (1865, 1705); (1865, 1994); (1865, 1695);
    (1865, 

There is a small problem.  We have a `int list list` and want a `int list`.  `flatten` to the rescue!

In [45]:
let l4 = List.flatten l3

val l4 : (int * int) list =
  [(1865, 1865); (1865, 1179); (1865, 1328); (1865, 1390); (1865, 322);
   (1865, 1999); (1865, 1713); (1865, 1808); (1865, 1380); (1865, 1727);
   (1865, 1702); (1865, 1304); (1865, 1481); (1865, 1334); (1865, 1728);
   (1865, 1953); (1865, 1413); (1865, 1753); (1865, 1723); (1865, 1379);
   (1865, 1532); (1865, 1918); (1865, 1490); (1865, 71); (1865, 1388);
   (1865, 1519); (1865, 807); (1865, 1427); (1865, 1729); (1865, 1952);
   (1865, 970); (1865, 1405); (1865, 1500); (1865, 1782); (1865, 1899);
   (1865, 1501); (1865, 1720); (1865, 1832); (1865, 1706); (1865, 1658);
   (1865, 1333); (1865, 486); (1865, 1670); (1865, 1674); (1865, 1290);
   (1865, 1893); (1865, 1382); (1865, 1761); (1865, 1945); (1865, 1607);
   (1865, 1319); (1865, 1508); (1865, 1464); (1865, 1733); (1865, 1917);
   (1865, 1483); (1865, 1620); (1865, 1677); (1865, 1835); (1865, 1810);
   (1865, 1385); (1865, 1840); (1865, 1705); (1865, 1994); (1865, 1695);
   (1865, 1599); (1865, 1681)

### 4: Find a tuple that sums to 2020

Instead of `List.filter` we'll use `List.find` here.  Instead of returning a list, `find f l` returns the first element in `l` that `f` returns `true` on.

In [46]:
let tuple_check (a,b) = a+b == 2020

val tuple_check : int * int -> bool = <fun>


In [47]:
let special_tuple = List.find tuple_check l4

val special_tuple : int * int = (71, 1949)


### 5: Multiply the tuple

`fst` and `snd` are used to access the components of the tuple.

In [48]:
(fst special_tuple) * (snd special_tuple)

- : int = 138379


# Advanced material

Below here is some advanced material you might be interested in, but we don't have time to cover.

## Variables

There are two types in variables in OCaml:
- Immutable
- Mutable

Functional programming is based on immutable variables, so we'll explore the let binding, which allows us to bind an expression to a variable.


In [49]:
"Hello" ^ " World!"

- : string = "Hello World!"


In [50]:
let s1 = "Hello" in
let s2 = " world!" in
s1 ^ s2

- : string = "Hello world!"


In [51]:
let discount = 0.1 in
let remain = 1.0 -. discount in
let msrp = 50000.0 in
msrp *. remain

- : float = 45000.


Variables in procedural languages are generally _mutable_, which means that they can hold different values at different times.  In OCaml, this is accomplished with a `ref` (reference) type.  As with `list` types, `ref` types take an argument that comes first, and describes the type of the object being stored.  For example, `int ref` is the type of a mutable integer, and `int list ref` is the type of a mutable list of integers.

`ref e` creates a reference initially to `e`.  To give a name to a reference variable, we usually use a let.  To access the current value of `v` we use `!v`.

In [52]:
let v = ref 42 in
!v

- : int = 42


In [53]:
let v = ref 42 in
v := 43;
!v

- : int = 43
